In [1]:
import minsearch
import json
from openai import OpenAI

In [2]:
with open('documents.json') as f_in:
    docs_raw = json.load(f_in)  

documents = []

for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc["course"] = course_dict["course"]
        documents.append(doc)

In [3]:
prompt_template = """
You're a course teaching assistant. 
Answer the QUESTION based on the context from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, ouput None.

QUESTION: {question}
CONTEXT: {context}
""".strip()

In [4]:
index = minsearch.Index(
                text_fields=["question", "text", "section"], 
                keyword_fields=["course"])
index.fit(documents)

In [5]:
client = OpenAI()

In [6]:
def search(query, index):
    boost = {'question': 3.0, 'section':0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results

In [7]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. 
Answer the QUESTION based on the context from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, ouput None.

QUESTION: {question}
CONTEXT: {context}
""".strip()
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [8]:
def llm(prompt):
    response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [10]:
query = "Can I follow the course once it finishes?"

In [11]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [12]:
from elasticsearch import Elasticsearch

In [13]:
es_client = Elasticsearch('http://localhost:9200')

In [15]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [16]:
from tqdm.auto import tqdm

In [18]:
for doc in tqdm(documents):
    es_client.index(index=index_name, body=doc)

In [19]:
query = "I just discovered the course, can I still join?"

In [29]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [30]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and 

In [33]:
def rag(query):
    results = elastic_search(query)
    prompt = build_prompt(query, results)
    response = llm(prompt)

    return response

In [34]:
response = rag(query)

In [35]:
print(response)

Yes, you can still join the course even if you have just discovered it. You are eligible to submit the homework assignments, but be aware that there will be deadlines for turning in the final projects.
